In [1]:
!pip install z3-solver
from z3 import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53.8 MB 154 bytes/s 


Estado inicial:

$$
\mathit{pc} = 0 \wedge (x=a) \wedge (y=b) \wedge (z=0) 
$$

As transições possíveis no FOTS são caracterizadas pelo seguinte predicado:

$$
\begin{array}{c}
(\mathit{pc} = 0 \wedge y = 0 \wedge \mathit{pc}' = 3 \wedge x' = x \wedge y' = y \wedge z' = z )\\
\vee\\
(\mathit{pc} = 0 \wedge (y\%2=0 \wedge y \neq 0) \wedge \mathit{pc}' = 1 \wedge x' = x \wedge y' = y \wedge z' = z)\\
\vee\\
(\mathit{pc} = 1 \wedge (2x ≤ 2^n-1) \wedge \mathit{pc}'= 0 \wedge x' = 2x \wedge y' = y/2 \wedge z' = z)\\
\vee\\
(\mathit{pc} = 0 \wedge (y\%2\neq0 \wedge y \neq 0) \wedge \mathit{pc}' = 2 \wedge x' = x \wedge y' = y \wedge z' = z)\\
\vee\\
(\mathit{pc} = 2 \wedge (z+x \le 2^n-1)\wedge \mathit{pc}'= 0 \wedge x' = x \wedge y' = y-1 \wedge z' = z+x)\\
\vee\\
(\mathit{pc} = 1 \wedge (2x > 2^n-1) \wedge \mathit{pc}'= 4 \wedge x' = 2x \wedge y' = y/2 \wedge z' = z)\\
\vee\\
(\mathit{pc} = 2 \wedge (z+x > 2^n-1) \wedge \mathit{pc}'= 4 \wedge x' = x \wedge y' = y-1 \wedge z' = z+x)\\
\\
\end{array}
$$

In [2]:
def declare(i):
    state = {}
    state['pc'] = Int('pc' + str(i))
    state['x'] = Const('x' + str(i), BitVecSort(n))
    state['y'] = Const('y' + str(i), BitVecSort(n))
    state['z'] = Const('z' + str(i), BitVecSort(n))
    return state 
      
def init(state):    
    return And(state['pc'] == 0, state['x'] == BitVecVal(a,n), state['y'] == BitVecVal(b,n), state['z'] == BitVecVal(0,n)) 

def trans(c,p):
    t03 = And(c['pc']==0, c['y']==BitVecVal(0,n), p['pc']==3, p['x']==c['x'], p['y']==c['y'], p['z']==c['z'])

    t01 = And(c['pc']==0, And(c['y']%2==0, c['y']!=0), p['pc']==1, p['x']==c['x'], p['y']==c['y'], p['z']==c['z'])
    t10 = And(c['pc']==1, BV2Int(c['x'])*2 <= (2**n)-1, p['pc']==0, p['x']==c['x']*2, p['y']==c['y']/2, p['z']==c['z'])

    t02 = And(c['pc']==0, And(c['y']%2!=0, c['y']!=0), p['pc']==2, p['x']==c['x'], p['y']==c['y'], p['z']==c['z'])
    t20 = And(c['pc']==2, BV2Int(c['z'])+BV2Int(c['x']) <= (2**n)-1, p['pc']==0, p['x']==c['x'], p['y']==c['y']-1, p['z']==c['z']+c['x'])

    t14 = And(c['pc']==1, BV2Int(c['x'])*2 > (2**n)-1, p['pc']==4, p['x']==c['x']*2, p['y']==c['y']/2, p['z']==c['z'])
    t24 = And(c['pc']==2, BV2Int(c['z'])+BV2Int(c['x']) > (2**n)-1, p['pc']==4, p['x']==c['x'], p['y']==c['y']-1, p['z']==c['z']+c['x']) 
    return Or([t03, t01, t02, t14, t24, t10, t20])

def inv(state):
    return state['x']*state['y']+state['z'] == a*b


In [3]:
def gen_trace(declare,init,trans,k):

    s = Solver()
    trace = [declare(i) for i in range(k)]

    s.add(init(trace[0]))

    for i in range(k-1):
      s.add(trans(trace[i],trace[i+1]))

    if s.check() == sat:
        m = s.model()
        for i in range(k):
            print(i)
            print('pc = ', m[trace[i]['pc']])
            print('x = ', m[trace[i]['x']])
            print('y = ', m[trace[i]['y']])
            print('z = ', m[trace[i]['z']])

In [4]:
def kinduction(declare,init,trans,inv,k):
    
    trace = [declare(i) for i in range(k+1)]
    
    s = Solver()
    
    s.add(init(trace[0]))
    
    for i in range(k-1):
        s.add(trans(trace[i],trace[i+1]))
    
    s.add(Or([Not(inv(trace[i])) for i in range(k)]))
    
    
    r = s.check()
    if r == sat:
        m = s.model()
        print("A proposição falha no caso base")
        for x in range(k):
            for v in trace[x]:
                print(v,"=",m[trace[x][v]])
        return
    if r!=unsat:
        return
    
    #provar o passo indutivo
    s = Solver()
    
    s.add(init(trace[0]))
    
    for i in range(k):
        s.add(trans(trace[i],trace[i+1]))
        s.add(inv(trace[i]))
    
    s.add(Not(inv(trace[k])))
    
    r = s.check()
    
    if r == sat:
        m = s.model()
        print("A proposição falha no passo k-indutivo que começa em")
        for v in trace[0]:
            print(v,"=",m[trace[0][v]])
        return
    if r ==unsat:
        print("O invariante verifica-se")
        return 


In [ ]:
n = 4 
a = 2 
b = 1 
k = 4
gen_trace(declare,init,trans,k)
kinduction(declare,init,trans,inv,k)

In [ ]:
n = 4
a = 0
b = 3
gen_trace(declare,init,trans,8)
kinduction(declare,init,trans,inv,8)

In [ ]:
n = 4
a = 3
b = 0
gen_trace(declare,init,trans,2)
kinduction(declare,init,trans,inv,2)

In [ ]:
n = 11
a = 20
b = 60
gen_trace(declare,init,trans,20)
kinduction(declare,init,trans,inv,20)

In [ ]:
n = 7
a = 40
b = 5
gen_trace(declare,init,trans,7)
kinduction(declare,init,trans,inv,7)